In [2]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Programa para buscar processos judiciais no nome de deputados federais eleitos em 2014 - depois filtra apenas por APs
#

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unidecode
import matplotlib.pyplot as plt
import re
%matplotlib inline

Só eleitos em 2014

In [3]:
eleitos_d = pd.read_csv('/home/reinaldo/Documentos/Code/RB2/candidatos/repo/resultados/dep_eleitos_2014_final.csv',sep=',',encoding = 'utf-8', converters={'sequencial': lambda x: str(x), 
                                                                                                                  'cpf': lambda x: str(x),  
                                                                                                                  'deputado_id': lambda x: str(x)})

In [4]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 15 columns):
uf                   513 non-null object
cargo                470 non-null object
nome_completo        513 non-null object
sequencial           513 non-null object
cpf                  513 non-null object
nome_urna            513 non-null object
partido_eleicao      513 non-null object
situacao_urna        513 non-null object
condicao             513 non-null object
deputado_id          513 non-null object
legislatura          513 non-null int64
partido_atual        513 non-null object
situacao_mandato     513 non-null object
uri                  513 non-null object
bancada_ruralista    513 non-null object
dtypes: int64(1), object(14)
memory usage: 60.2+ KB


In [5]:
def f(str):
    return (unidecode.unidecode(str))

In [6]:
eleitos_d["nome_completo"] = eleitos_d["nome_completo"].apply(f)

In [7]:
def le_lista(politico_link, nome_urna, nome, cpf):
    print(nome_urna)
    try:
        res = requests.get(politico_link)
    except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
        print(str(e))
        return 
    except Exception as e:
        print("Exceção")
        return
        
    soup =  BeautifulSoup(res.text, "lxml")

    parts = soup.find_all('table', {'class': 'table m-b-0'})
    
    for part in parts:
        lines = part.find_all('td')
        
        chunks = [lines[x:x+5] for x in range(0, len(lines), 5)]
    
        for vez in chunks:
            processos.append({'nome_completo': nome, 
                              'cpf': cpf, 
                                'nome_urna': nome_urna, 
                                'identificacao': vez[0].string, 
                                'identificacao_link': "http://portal.stf.jus.br/processos/" + vez[0].a.get('href'), 
                                'numero_unico': vez[1].string, 
                                'data_autuacao': vez[2].string, 
                                'meio': vez[3].string, 
                                'publicidade': vez[4].string})    
 
    return processos

In [8]:
link_stf = 'http://portal.stf.jus.br/processos/listarPartes.asp?termo='
    
processos = []
    
for num, row in eleitos_d.iterrows():
    nome = row['nome_completo']
    nome_urna = row['nome_urna']
    cpf = row['cpf']
    politico_link = link_stf + nome
    le_lista(politico_link, nome_urna, nome, cpf)

ABEL GALINHA
ADILTON SACHETTI
ELI CORREA FILHO
AELTON FREITAS
AFONSO MOTTA
ALAN RICK
ALCEU MOREIRA
ALEX CANZIANI
ALEXANDRE BALDY
ALEXANDRE LEITE
ANDRE ABDON
ANDRE FUFUCA
ANIBAL
BALMAN
JACOME
IMBASSAHY
TONINHO PINHEIRO
ROBERTO GOES
SERGIO VIDIGAL
JORDY
ARTHUR LIRA
ARTHUR MAIA
ASSIS DO COUTO
ATILA LIRA
AUGUSTO CARVALHO
AUREO
BENITO GAMA
BENJAMIN MARANHAO
BONIFACIO ANDRADA
BRUNA FURLAN
CAIO NARCIO
BETINHO ROSADO SEGUNDO
CESAR MESSIAS
CARLOS MELLES
MARUN
CARLOS BEZERRA
CARLOS GAGUIM
MANATO
CELIO SILVEIRA
CELSO MALDANER
CELSO PANSERA
CESAR HALUM
CLAUDIO CAJADO
CLEBER VERDE
CRISTIANE BRASIL
DAGOBERTO NOGUEIRA
DANIEL VILELA
PERONDI
DIEGO GARCIA
DIEGO ANDRADE
DILCEU SPERAFICO
DOMINGOS NETO
DOMINGOS SAVIO
DULCE MIRANDA
DELEGADO EDER MAURO
EDIO LOPES
EDMAR ARRUDA
EDUARDO CUNHA
EFRAIM FILHO
ELCIONE
ESPERIDIAO AMIN
EVAIR DE MELO
EVANDRO GUSSI
EVANDRO ROMAN
EZEQUIEL FONSECA
EZEQUIEL TEIXEIRA
FABINHO RAMALHO
FABIO GARCIA
FAUSTO PINATO
FELIPE BORNIER
FELIX JR
FERNANDO JORDAO
GIACOBO
FLAVIA MORAIS
CHA

In [9]:
df_processos = pd.DataFrame(processos, columns = ['nome_completo', 'cpf', 'nome_urna', 'identificacao','identificacao_link', 'numero_unico','data_autuacao', 'meio', 'publicidade'])

In [10]:
df_processos.reset_index().head()

,index,nome_completo,cpf,nome_urna,identificacao,identificacao_link,numero_unico,data_autuacao,meio,publicidade
0,0,ADILTON DOMINGOS SACHETTI,45360707968,ADILTON SACHETTI,Pet 3473,http://portal.stf.jus.br/processos/detalhe.asp...,0003235-67.2005.0.01.0000,26/07/2005,Físico,Público
1,1,ADILTON DOMINGOS SACHETTI,45360707968,ADILTON SACHETTI,Rcl 5863,http://portal.stf.jus.br/processos/detalhe.asp...,0000741-30.2008.0.01.0000,25/02/2008,Físico,Público
2,2,ADILTON DOMINGOS SACHETTI,45360707968,ADILTON SACHETTI,AP 947,http://portal.stf.jus.br/processos/detalhe.asp...,0003709-32.2015.1.00.0000,17/06/2015,Físico,Público
3,3,ADILTON DOMINGOS SACHETTI,45360707968,ADILTON SACHETTI,Inq 4165,http://portal.stf.jus.br/processos/detalhe.asp...,0008572-31.2015.1.00.0000,17/11/2015,Físico,Público
4,4,ADILTON DOMINGOS SACHETTI,45360707968,ADILTON SACHETTI,Inq 4210,http://portal.stf.jus.br/processos/detalhe.asp...,0043212-26.2016.1.00.0000,10/03/2016,Físico,Público


In [11]:
df_processos.to_excel('resultados/processos_stf_deputados_14_jul_2018.xlsx',sheet_name='Sheet1')

Seleciona apenas Ações Penais

In [12]:
mask = df_processos['identificacao'].str.contains("AP ", na=False)
aps = df_processos[mask]

In [13]:
aps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 2 to 9961
Data columns (total 9 columns):
nome_completo         157 non-null object
cpf                   157 non-null object
nome_urna             157 non-null object
identificacao         157 non-null object
identificacao_link    157 non-null object
numero_unico          157 non-null object
data_autuacao         157 non-null object
meio                  157 non-null object
publicidade           157 non-null object
dtypes: object(9)
memory usage: 12.3+ KB


In [14]:
aps.to_excel('resultados/processos_AP_stf_deputados_14_jul_2018.xlsx',sheet_name='Sheet1')